### Day 5 - Learnings

I saw Mark's awesome solution: https://github.com/mark-ferguson-kroger/aoc-2020/blob/main/aoc5.py

Very clean, very efficient! Wanted to understand what was happening.

#### My original solution

- I "manually" did division & updated either min or max based on the letter received. This gets clunky & requires looping and applying a function to do some unecessary math.

#### Solving with Binary

The code above converts all of the input data to binary, so will break down for myself what is happening. 

We can take an example provided us -  `FBFBBFFRLR:`

```
F means take the lower half
B means take the upper half
```

#### Base 2 vs Base 10

Binary / base 2 has a different meaning than decimal / base 10. 

In base 10 each digit in a number represents 1 of 10 sections (`values of 0-9`). 
For example `056` tells us: 
- `0`: We are in the 000s. 
- `5`: We are in the 50s. 
- `6`: We are in the 10s, but this actually provides our actual number. 

base 2 does the same, but each digit is going to represent 1 of 2 sections (`values of 0-1`), which makes it a great candidate for splitting in half!

For example `010`: 
- Firstly, since this is three digits this number can only be 0 - 7 (`2*3` = `8` possible values)
- `Leftmost 0`: This represents numbers 0-3. If it were 1 then this would be 4-7. 
- `Middle 1`: This value changes based on the value to left. 
    - If left were 0 & this is 1 then we have options of: `2-3`
    - If left were 1 & this is 1 then we have options of: `3-4`
- `Rightmost 0`: This is our last value & determines our number. In this case we take the lower bound, which yields `2`.

Full binary breakdown: 

```
000 = 0
001 = 1
010 = 2
011 = 3
100 = 4
101 = 5
110 = 6
111 = 7
```

#### Using the above for solving the problem: 

Let's return to our test case:  `FBFBBFF`
- Min row = 0
- Max row = 127
- Expected row: `44`

Breaking it down from left to right: 
- `F` -> 0: 0 - 63
- `B` -> 1: 32 - 63
- `F` -> 0: 32 - 47
- `B` -> 1: 40 - 47
- `B` -> 1: 44 - 47
- `F` -> 0: 44 - 45
- `F` -> 0: 44

#### Python code below: 

In [1]:
# test case
test_case = "FBFBBFF"

# convert to binary string by replacing letters properly 
test_case = test_case.replace("F", "0").replace("B", "1")

# we can then convert this to an int with base-2, which will yield our row
print(f"Converted to binary: {int(test_case,2)}")

#### But it gets better: 

- We can actually solve the full problem, which asks us to get the `row` (first 7 chars) and `col` (last 3 chars), then solve for `id = row * 8 + col`
- Details for col:
```
L means take the lower half
R means take the upper half
```

Binary will allow us to handle all of this due to that `*8` off the col. 
- This means we are expanding our digits out by 3....the same as 2 x 2 x 2, or 8 (I could make this clearer). 
- 7 digit binary x 8 is the same as a 10 digit binary:
    `1011100 * 8` = 736
    `1011100000` = 736

Mark was able to bypass that `id` calculation by recognizing this & just solving for a 10-digit binary (instead of doing this in 2 chunks and applying equation above). 


In [2]:
# test case -> ID should be 357
test_case = "FBFBBFFRLR"

# convert to binary string by replacing letters properly 
test_case = test_case.replace("F", "0").replace("B", "1").replace("L", "0").replace("R", "1")

# we can then convert this to an int with base-2, which will yield our row
print(f"Our ID is: {int(test_case,2)}")

Our ID is: 357


In [3]:
# now we can write a simple function to solve for all ids: 
def determineID(idString):
    """Function to convert input string into ID value based on base-2"""
    return int(idString.replace("F", "0").replace("B", "1").replace("L", "0").replace("R", "1"),2)

In [4]:
# confirm all shared test cases
assert(determineID("FBFBBFFRLR") == 357)
assert(determineID("BFFFBBFRRR") == 567)
assert(determineID("FFFBBBFRRR") == 119)
assert(determineID("BBFFBBFRLL") == 820)

#### Solving Part 1 & 2 now: 

- much more efficient!

In [7]:
# Read in data
with open('day05.txt') as fh:
    lines = fh.readlines()

text = [line.strip() for line in lines]

id_list = []

# iterate through each case, solve for ID & append
for case in text:
    id_list.append(determineID(case))

# Sorting IDs
id_list.sort()

# iterate over id_list until we find a gap 
for i, id_val in enumerate(id_list):
    if id_val + 1 != id_list[i+1]:
        missing_id = id_val + 1
        break
        
print(f"Max ID is: {id_list[-1]}")
print(f"My ID is: {missing_id}")

Max ID is: 987
My ID is: 603
